In [435]:
import bs4
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pickle
import re
import pandas as pd
import matplotlib.pyplot as plt
import wikipediaapi
import requests
import time
import numpy as np
import pickle
import csv
import scipy as sp
from IPython.display import clear_output
import wptools
import wikipedia
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from nltk.collocations import *
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import string
import itertools as it
import json
# For a nice view of the output
from IPython.display import HTML, display
from tabulate import tabulate
import num2words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dilet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dilet\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dilet\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [436]:
movies1=pd.read_csv('movies1.tsv', sep = '\t')
movies3=pd.read_csv('movies3.tsv',sep='\t')

In [437]:
movies3link=pd.read_html('movies3.html')
movies3link=movies3link[0]
movies3link=movies3link['URL'].tolist()

In [438]:
movies3['wikipedia_url']=0

In [439]:
for i in range(len(movies3)):
    movies3['wikipedia_url'][i]=movies3link[i]

D:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [440]:
df=pd.concat([movies1,movies3], ignore_index=True,sort=False)

In [441]:
df.columns

Index(['Unnamed: 0', 'title', 'intro', 'plot', 'film_name', 'director',
       'producer', 'writer', 'starring', 'music', 'release_date', 'runtime',
       'country', 'language', 'budget', 'plot_s', 'wikipedia_url', 'story'],
      dtype='object')

In [443]:
df.drop(['Unnamed: 0', 'plot_s','story'], axis=1, inplace = True)

In [448]:
df=df[0:500]

In [31]:
#new_df=df[['title','intro','plot','wikipedia_url']]

In [450]:
len(df.columns)

15

In [455]:
df['plot'][467:469]

467    André-Louis Moreau (Ramon Novarro) loves Aline...
468    In 1906, Wilse Dilling (Lon Chaney), a cripple...
Name: plot, dtype: object

In [278]:
#Read the file and create a single tsv file for each record.  
for i in range(len(df)):
    file = open(r'C:\Users\Dilet\Desktop\Homework_3\data\Part_1\doc_' + str(i+1) 
              + '.tsv','w', newline = '', encoding = 'utf-8') 
    for j in range(len(df.columns)): #is the number of columns
        file.write('%s\t' %df.iloc[i,j])
    file.close()

**Details:**
- We store all the documents into an empty folder called *'Doc_i'* and this folder is created especially for them;
- str(i+1) : **i** is the dataframe index but we increase it onefold because in thi way the numbering of the documents start from 1 and not from 0;
- The name of each file is *doc_i.tsv*.

For example, if we open **doc_2.tsv**, the content of the file should be the following:
'The Martyred Presidents	The Martyred Presidents is a 1901 American film directed by Edwin S. Porter.	The film, just over a minute long, is composed of two shots. In the first, a girl sits at the base of an altar or tomb, her face hidden from the camera. At the center of the altar, a viewing portal displays the portraits of three U.S. Presidents—Abraham Lincoln, James A. Garfield, and William McKinley—each victims of assassination.
In the second shot, which runs just over eight seconds long, an assassin kneels feet of Lady Justice.	The Martyred Presidents	[[Edwin S. Porter]]	nan	nan	See below	nan	nan	1 minutes	United States	English	nan	https://en.wikipedia.org/wiki/The_Martyred_Presidents	'

# 2. Search Engine
### As a first common step, you must preprocess the documents by
### 1. Removing stopwords
### 2. Removing punctuation
### 3. Stemming
### 4. Anything else you think it's needed




# METHODS

## Step 3: Search Engine
As a first common step, we preprocess the documents by:
- **Removing stopwords:** if there are words that don’t really have any meaning to our text we can just remove them. This words have sense in our language but in data analysis they are useless;
- **Removing punctuation:** ! # $ % & \ ' () * + , - . / : ; < = > ? @ [ \\ ] ^ _ ` {|} ~
- **Stemming:** the idea is take from the word the root stem of the word. We do this because for a lot of items we are going to have different variations of word based on their stems or at least sort of fixes at the end but really the actual meaning of that word is unchanged.
- **Lemmatizing:** is a very similar operation to stemming but the end result is a real word, so, the end result will be generally some form of synonym or it might be the original word. So, you might end up with a very different word but it will be a similar word with the same meaning;


We know that lemmatization may have accept only words that are part of the English dictionary so we delete all the words of other languages.e been a risky choice. In fact, using lemmatizing the answer to the query is larger than when it is not used. We have chosen to do this operation because the result could be more satisfactory for the customer. We show our reasoning with an ***example***:

In English (and most languages), words appear in different forms. Look at these two sentences:

`A garden with cactus` -> lemmatizer.lemmatize("cactus")) return **cactus**

`A garden with cacti`  -> lemmatizer.lemmatize("cacti") return **cactus**

Both sentences talk about the noun cactus, but they are using different inflections. When working with text in a computer, it is helpful to know the base form of each word so that you know that both sentences are talking about the same concept. Otherwise the strings “cactus” and “cacti” look like two totally different words to a computer.


In [281]:
# Normalize the word. i.e. tokenize,stemming, lemmatization and removing punctuations
exclude = set(string.punctuation)
def wordNormlization(text):
    stemmer = SnowballStemmer("english")
    lemmatizer = WordNetLemmatizer()
    Normalized = []
    tokenizer = RegexpTokenizer(r'\w+')
    word_list = tokenizer.tokenize(text.lower())
    Remove_stopwords = [stemmer.stem(lemmatizer.lemmatize(word)) 
                        for word in word_list if word not in stopwords.words('english')]
    for word in Remove_stopwords:
        if not word.isdigit() and len(word)>1:
            word = ''.join(ch for ch in word if ch not in exclude) #remove the punctuation
            Normalized.append(word)
    for word in Remove_stopwords:
        if word.isdigit():
            word= num2words.num2words(int(word), lang='eng')
            Normalized.append(word)
    return Normalized

### 2.1) Conjunctive query
At this moment, we narrow out interest on the **intro** and **plot** of each document.

Now, what we do is:

1) `Read` all the documents inside the folder which contains all the 30000 tsv files;

2) `Preprocess` the discription and title for each file;

3) Create a `list` of all the words "clean";

4) Create a `dictionary` whose key is the name of the document and the words contained in it for the items.

In [282]:
words = []
dict_ = {}
for i in range(len(df)):
    file = open(r'C:\Users\Dilet\Desktop\Homework_3\data\Part_1\doc_'+ str(i+1) 
              + '.tsv','r', encoding = 'utf-8') 
    file_name = 'doc_'+ str(i+1) + '.tsv' 
    list_ = file.read().split('\t')
    intro_and_plot = list_[1] + list_[2] #intro 1 plot 2 colum
    cleaning = wordNormlization(str(intro_and_plot))
    words.append(cleaning)
    dict_[file_name] = cleaning
    file.close()
words = list(it.chain(*words)) # Convert a nested list into one-dimensional list

We also save the list of all the word inside a txt file

In [189]:
with open(r'C:/Users/Dilet/Desktop/Homework_3/data/Part_2/file/listwords.txt', 'w') as filehandle:  
       json.dump(words, filehandle)

with open(r'C:/Users/Dilet/Desktop/Homework_3/data/Part_2/file/listwords.txt','r') as filehandle:  
    words = json.load(filehandle)

**CreateVocabulary(words)**: this function take in input all words, create the set and assign an ID to each term.

*Vocabulary = {term : termID}*

In [283]:
def CreateVocabulary(words):
    # vocabulary = {term : termID}
    Vocabulary = {}
    wordList = list(set(words))
    for ID, element in enumerate(wordList):
        Vocabulary.update({element : ID})
    return Vocabulary

In [284]:
voc = CreateVocabulary(words)

If you are interested in seeing the id associated with each word, open the file `'vocabulary.txt'` which is in the GitHub repository.

In [190]:
# Lets save the vocabulary for future use
with open(r'C:/Users/Dilet/Desktop/Homework_3/data/Part_2/file/vocabulary.txt', 'w') as filehandle:  
    json.dump(voc, filehandle)

with open(r'C:/Users/Dilet/Desktop/Homework_3/data/Part_2/file/vocabulary.txt', 'r') as filehandle:  
    voc =  json.load( filehandle)

In [285]:
len(voc) #total number of the word

8922

In [286]:
#Build invertedIndex
# invertedIndex = {termID : doc}
def invertedIndex(vocabulary):
    invertedIndex= {}
    for key,value in vocabulary.items():
        for i in value:
            if (voc[i] in invertedIndex):
                invertedIndex[voc[i]] = invertedIndex[voc[i]] + [key]
            else:
                invertedIndex[voc[i]] = [key]
    return invertedIndex

In [287]:
invertedDic = invertedIndex(dict_)

In [288]:
invertedDic

{7853: ['doc_1.tsv',
  'doc_25.tsv',
  'doc_25.tsv',
  'doc_25.tsv',
  'doc_25.tsv',
  'doc_25.tsv',
  'doc_26.tsv',
  'doc_28.tsv',
  'doc_28.tsv',
  'doc_28.tsv',
  'doc_32.tsv',
  'doc_33.tsv',
  'doc_33.tsv',
  'doc_33.tsv',
  'doc_34.tsv',
  'doc_34.tsv',
  'doc_38.tsv',
  'doc_39.tsv',
  'doc_40.tsv',
  'doc_49.tsv',
  'doc_51.tsv',
  'doc_58.tsv',
  'doc_61.tsv',
  'doc_71.tsv',
  'doc_74.tsv',
  'doc_78.tsv',
  'doc_79.tsv',
  'doc_79.tsv',
  'doc_80.tsv',
  'doc_80.tsv',
  'doc_92.tsv',
  'doc_92.tsv',
  'doc_97.tsv',
  'doc_101.tsv',
  'doc_102.tsv',
  'doc_102.tsv',
  'doc_103.tsv',
  'doc_104.tsv',
  'doc_105.tsv',
  'doc_105.tsv',
  'doc_107.tsv',
  'doc_110.tsv',
  'doc_110.tsv',
  'doc_112.tsv',
  'doc_121.tsv',
  'doc_122.tsv',
  'doc_126.tsv',
  'doc_128.tsv',
  'doc_131.tsv',
  'doc_131.tsv',
  'doc_132.tsv',
  'doc_132.tsv',
  'doc_133.tsv',
  'doc_134.tsv',
  'doc_136.tsv',
  'doc_138.tsv',
  'doc_143.tsv',
  'doc_146.tsv',
  'doc_146.tsv',
  'doc_146.tsv',
  'doc_1

If you are interested in seeing the inverted index, open the file `'inverted.json'` 

In [231]:
#Save our invertedIndex file for future use.
with open(r'C:/Users/Dilet/Desktop/Homework_3/data/Part_2/file/inverted.json', 'w') as fp:
    json.dump(invertedDic, fp)
with open(r'C:/Users/Dilet/Desktop/Homework_3/data/Part_2/file/inverted.json','r') as f:
    invertedDic = json.load(f)

### 2.1.2) Execute the query
Given a query, that we let the user enter.

In [418]:
query = str(input().split()) #input the word do you want

disney 2019


In [419]:
qnorm = wordNormlization(query) #Normalize the query

In [420]:
qnorm #result

['disney', 'two thousand and nineteen']

**What documents do we want?**
Since we are dealing with conjunctive queries (***AND***), each of the returned documents should contain all the words in the query. 
`AND <--> intersection`

We use set intersection to find the query words in all the documents. Our approach is:

- **First,** we store the documents that contain the first word.
- **Second,** we check the intersection of the first word with the second word. If we found that there is a common documents for each word, we store the common files into `inter` set and continue to find the intersection with the next word. In the meantime, if we don't find any intersection with the words; we stop looking for the document that contain other words. 

In [421]:
inter = set() #set with the documents that contain all the words in the query
switch = True
for word in qnorm:
    if word in voc:
        if (switch):
            inter = inter.union(invertedDic[voc[word]])
            switch = False
        if(len(inter) >0):
            
                inter = inter.intersection(set(invertedDic[voc[word]]))
        else:
            break

Show what are all the documents that contain all the words contained in the query:

In [422]:
inter

{'doc_465.tsv',
 'doc_466.tsv',
 'doc_467.tsv',
 'doc_468.tsv',
 'doc_473.tsv',
 'doc_474.tsv'}

In [426]:
df.head(2)

,title,intro,plot,film_name,director,producer,writer,starring,music,release_date,runtime,country,language,budget,wikipedia_url
0,Love by the Light of the Moon,Love by the Light of the Moon is a 1901 film b...,"The moon, painted with a smiling face hangs ov...",Love by the Light of the Moon,[[Edwin S. Porter]],NaN,NaN,NaN,NaN,NaN,NaN,United States,Silent film,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...
1,The Martyred Presidents,The Martyred Presidents is a 1901 American fil...,"The film, just over a minute long, is composed...",The Martyred Presidents,[[Edwin S. Porter]],NaN,NaN,See below,NaN,NaN,1 minutes,United States,English,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...


### Final output of the query

In [456]:
my_dataframe = pd.DataFrame()
for file in inter:
    table = pd.read_table(r'C:/Users/Dilet/Desktop/Homework_3/data/Part_1/'+ str(file),sep='\t',
                          header = None,index_col = None,   error_bad_lines=False)
    my_dataframe = my_dataframe.append(pd.DataFrame(table))

    my_dataframe = my_dataframe.rename(columns={0: 'title', 1:'intro', 2:'plot',
                  3:'film_name',4:'director',5:'producer',
                                                6:'writer',7:'starring',8:'music', 9:'release_date'
                                      ,9:'runtime', 10:'country', 11:'language', 12:'budget', 13:'wikipedia_url'})

df2 = df[['title','intro','wikipedia_url']]
df2.drop_duplicates

display(HTML(tabulate(my_dataframe[[0,1,13]], headers=['TITLE','INTRO','URL WIKIPEDIA'],showindex=False, tablefmt='html' )))

b'Skipping line 3: expected 2 fields, saw 15\n'
b'Skipping line 6: expected 2 fields, saw 14\n'
b'Skipping line 6: expected 2 fields, saw 14\n'


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

# 2.2) Conjunctive query & Ranking score

### In the new Search Engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

### Find all the documents that contains all the words in the query. Sort them by their similarity with the query Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents. To solve this task, you will have to use the tfIdf score, and the Cosine similarity. Let's see how.

## 2.2.1) Inverted index